In [20]:
import pandas as pd
import sklearn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
X_train=pd.read_csv("train-new-2.csv", sep=';')
y_train=pd.read_csv("train-target-new-3.csv")
X_test=pd.read_csv("test-new-2.csv", sep=';')
d = dict.fromkeys(X_train.columns)
for j,i in enumerate(d.keys()):
    d[i]=j
X_train=X_train.rename(columns=d)
h = dict.fromkeys(X_test.columns)
for j,i in enumerate(h.keys()):
    h[i]=j
X_test=X_test.rename(columns=h)

In [46]:
x_train, x_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3)

In [47]:
scaler=StandardScaler()
sgd=SGDClassifier(penalty='l1', random_state=0)
clf = Pipeline([
  ('scaling', scaler),
  ('classification', sgd)
])
clf.fit(x_train, y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(memory=None,
         steps=[('scaling',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('classification',
                 SGDClassifier(alpha=0.0001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                               max_iter=1000, n_iter_no_change=5, n_jobs=None,
                               penalty='l1', power_t=0.5, random_state=0,
                               shuffle=True, tol=0.001, validation_fraction=0.1,
                               verbose=0, warm_start=False))],
         verbose=False)

In [48]:
params={
    'classification__alpha':[0.0001, 0.001, 0.01, 0.1, 1],
    'classification__loss': ['log']
}

In [50]:
grid_cv=GridSearchCV(clf, params, cv=4)
grid_cv.fit(x_train, y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

GridSearchCV(cv=4, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('scaling',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('classification',
                                        SGDClassifier(alpha=0.0001,
                                                      average=False,
                                                      class_weight=None,
                                                      early_stopping=False,
                                                      epsilon=0.1, eta0=0.0,
                                                      fit_intercept=True,
                                                      l1_ratio=0.15,
                                                      learning_rate='optimal',


In [51]:
grid_cv.best_params_

{'classification__alpha': 0.0001, 'classification__loss': 'log'}

In [52]:
best=grid_cv.best_estimator_

In [54]:
roc_auc_score(y_test, best.predict_proba(x_test)[:,1])

1.0

In [57]:
score=pd.DataFrame(best.predict_proba(X_test)[:,1])
score.to_csv(r'score_LR10.csv', index=False, header=0)